Your first workchain
======

We are sorry to give you this responability right from the beggining, but **the world needs your ability to develop workchains**.

## Welcome to the project, here's the roadmap

As you might know, *climate change is a thing*. Climate change will not only affect our oceans, cities and forests, but also our simulations. Just recently, the EU released their roadmap to keep atomic simulations correct through this hard times. These are the four stages that need to be achieved:

<div id="stage1" style="border: solid 2px green; background-color: rgba(0,255,0,0.2); padding: 10px 20px; margin: 30px 10px; border-radius: 7px">
    
#### DEGRADATION OF THE KINETIC ENERGY
    
*Climate change will take its toll on the movement of electrons. This will result in a constant increase of the energy of any system, according to the laws of quantum mechanics. The offset is expected to follow Bosoni-Febrer's equation:* $offset(year) = year - 2020$.
    
*For this reason, it is of unprecedented importance that we are able to incorporate this offset into our calculations in order to predict future energies of structures.* 
    

<div style="margin-top: 10px;border-top: solid 1px green">
</div>
    
**Stage goal**: A workflow that runs SIESTA and returns the energy of the structure in the year requested by the user.
    
<div style="text-align: right; color:green"><b><i>Stage 1</i></b></div>
    
</div>

<div id="stage2" style="border: solid 2px green; background-color: rgba(0,255,0,0.2); padding: 10px 20px; margin: 30px 10px; border-radius: 7px">
    
#### ADAPTING TO EXTINTION
    
*According to scientists, at the current pace Carbon atoms will be extinguished by 2040. In general, carbon structures will see their C atoms changed to Si, but in some special cases they will be substituted by other atoms.*

<div style="margin-top: 10px;border-top: solid 1px green">
</div>
    
**Stage goal**: We need to incorporate the elimination of C atoms by 2040 in our calculations. By default, C atoms should be substituted by Si, but the user should be able to change that and provide a different atom. 
    
<div style="text-align: right; color:green"><b><i>Stage 2</i></b></div>
    
</div>

<div id="stage3" style="border: solid 2px orange; background-color: rgba(255,165,0,0.2); padding: 10px 20px; margin: 30px 10px; border-radius: 7px">
    
#### A MIXING MESS
    
*After the water wars (forecasted by 2025), radioactivity will be all over the place. By then, the behavior of atomic nuclei will be unpredictable. This will of course translate in more uncertainty in the final position of electrons when atoms interact. Therefore, the optimal mixing coefficients during SCF cycles will suffer great oscillations. Scientists will need to be constantly looking for the optimal value to run their simulations.*

<div style="margin-top: 10px;border-top: solid 1px orange">
</div>
    
**Stage goal**: A workflow that iterates the workchain designed at stage 2 over different `scfmixerweight` and returns the fastest one.
    
<div style="text-align: right; color:orange"><b><i>Stage 3</i></b></div>
    
</div>

<div id="stage4" style="border: solid 2px red; background-color: rgba(255,0,0,0.2); padding: 10px 20px; margin: 30px 10px; border-radius: 7px">
    
#### OPTIMIZING EXPERIMENT SPEED
    
*Climate change does not leave much margin for experimentation. We need to act fast. That's why we can not waste unnecessary time in simulations. All simulations need to be optimized to reach their result as soon as possible.*

<div style="margin-top: 10px;border-top: solid 1px red">
</div>
    
**Stage goal**: A workflow that, given an unrelaxed structure uses the stage 3 deliverable to get the value of `scfmixerweight` that results in the fastest simulation and then runs a CG relaxation with that parameter to get the relaxed structure.
    
<div style="text-align: right; color:red"><b><i>Stage 4</i></b></div>
    
</div>

## Some context
Here are the imports that you will probably need to develop your workchain:

In [ ]:
import aiida
from aiida.engine import WorkChain, ToContext, calcfunction
from aiida.orm import Code, Str, Dict, Float, StructureData
from aiida_siesta.workflows.base import SiestaBaseWorkChain
from aiida_siesta.workflows.iterate import SiestaIterator

import numpy as np

The *European Scientific Association* (ESA) prepared a python module so that you can submit your deliverables:

In [ ]:
import deliver
from deliver import deliver_stage, stage_solution

They will check whether the workchain you developed actually satisfies all the requirements. To submit the deliverable for stage 1, for example, do:

```python
deliver_stage(1, Stage1Deliverable)
```

<div class="alert alert-info">

PSSSST!
    
You can use `stage_solution(1)` to get the solution for stage 1, but **only if you are really hopeless**. Otherwise use the hints provided by each stage.
    
</div>

Now we need to load the profile in which our workchains will run:

In [ ]:
aiida.load_profile("your profile here")

And inform the deliver package what are the general inputs that you want to use to run your deliverables (If you are following the full tutorial in `aiida-siesta`, the values provided should be fine).

In [ ]:
deliver.GENERAL_INPUTS = {
    "code": Code.get_from_string('siesta-v4.1@localhost'),
    "pseudo_family": Str("nc-sr-04_pbe_standard-psf"),
    "options": Dict(
        dict={
            'withmpi': False,
            'max_wallclock_seconds': 3600 * 2
        }
    ),
    "parameters": Dict(),
}

**We're all set.** Good luck with developing!

Just kidding, the ESA gathered some of their best scientists to produce a template that you can use as the starting point for your deliverables:

In [ ]:
class ClimateAdaptedCalculation(WorkChain):
    
    @classmethod
    def define(cls, spec):
        super().define(spec)

        # Define the outline of the workflow, i.e. the order in which methods are executed.
        spec.outline(
            cls.initialize,
            cls.run_calc,
            cls.process_calc
        )

        # We expose the inputs of the SiestaBaseWorkChain so that they can be passed to our workchain
        spec.expose_inputs(SiestaBaseWorkChain, exclude=["metadata"])
        # Also, we expose its outputs so that they can be easily accessed from this workchain
        spec.expose_outputs(SiestaBaseWorkChain)
        
        # In case we need to define more inputs, we call spec.input
        spec.input("my_input")
        
        # Similarly, if we want to define outputs, we do it like this
        spec.output("my_output")
    
    def initialize(self):
        """
        Here we will do some initialization/preprocessing if we need it.
        """
        
    def process_inputs(self, inputs):
        """
        This is a chance to process the inputs that will go into the calculation.
        
        It's called before submitting the calculation (see run_calc).
        """
        return inputs
    
    def run_calc(self):
        """
        Where we actually run the calculation
        """
        # Get the inputs that we exposed for SiestaBaseWorkChain
        inputs = self.exposed_inputs(SiestaBaseWorkChain)
        
        # Process the inputs if we need to
        inputs = self.process_inputs(inputs)
        
        # Submit the calculation
        process_node = self.submit(SiestaBaseWorkChain, **self.ctx.siesta_inputs)

        # Wait for the process to finish before going to the next step
        # This will store the process into self.ctx
        return ToContext(calc=process_node)
    
    def process_calc(self):
        """
        Here we will process the outputs of the calculation
        """
        # Retrieve the process that we put to context at the end of run_calc
        process_node = self.ctx.calc
        
        # We return all the outputs from SiestaBaseWorkChain
        self.out_many(self.exposed_outputs(self.ctx.calc, SiestaBaseWorkChain))
        
        # Process the outputs to generate a new node (see process_output docs)
        my_output_value = process_output(process_node.outputs.output_parameters, )
        
        # When we are ready to output some value, we just need to call self.out 
        self.out("my_output", my_output_value)

@calcfunction
def process_output(output_parameters):
    """
    Processes the output and generates a new node.
    
    In aiida, when you generate a node (i.e. something that you want to use as
    an input or output), you have to do it inside a calcfunction.
    """
    return 

## Your time to shine

Now, let's get to work. Here you have all the cells prepared to receive your code!

**Stage 1** ([Go to description](#stage1))

In [ ]:
deliver_stage(1, Stage1Deliverable)

**Stage 2** ([Go to description](#stage1))

In [ ]:
deliver_stage(2, Stage2Deliverable)

**Stage 3** ([Go to description](#stage3))

In [ ]:
deliver_stage(3, Stage3Deliverable)

**Stage 4** ([Go to description](#stage4))

In [ ]:
deliver_stage(4, Stage4Deliverable)